<a href="https://colab.research.google.com/github/elikapati/DeepLearning/blob/master/MnistClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Classification of Images from Tensor dataset

In [0]:
# If required, install tensorflow datasets using the command:
#  !pip install -U tensorflow_datasets

from __future__ import absolute_import, division, print_function

# Import Tensorflow and Tensorflow Datasets
import tensorflow as tf
import tensorflow_datasets as tfds

# Import Helper functions
import math
import numpy as np
import matplotlib.pyplot as plt

# Import progressive bar display
import tqdm
import tqdm.auto

tqdm.tqdm = tqdm.auto.tqdm

print(tf.__version__)

# Not required for tensorflow 2.0
tf.enable_eager_execution()

In [0]:
dataset, metadata = tfds.load('fashion_mnist', as_supervised=True, with_info=True)
train_dataset, test_dataset = dataset['train'], dataset['test']

In [0]:
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

In [0]:
num_train_examples = metadata.splits['train'].num_examples
num_test_examples = metadata.splits['test'].num_examples

print("No. of training examples = {}".format(num_train_examples))
print("No. of testing examples = {}".format(num_test_examples))

In [0]:
# Preprocess data
def normalize(images, labels):
  images = tf.cast(images, tf.float32)
  images /= 255
  return images, labels

train_dataset = train_dataset.map(normalize)
test_dataset = test_dataset.map(normalize)

In [0]:
# Explore processed data
for image, label in test_dataset.take(1):
  break

image = image.numpy().reshape((28, 28))

# Plot image
plt.figure()
plt.imshow(image, cmap=plt.cm.binary)
plt.colorbar()
plt.grid(False)
plt.show()

In [0]:
# Plot 25 images
plt.figure(figsize=(10, 10))
i = 0
for (image, label) in test_dataset.take(25):
  image = image.numpy().reshape((28, 28))
  plt.subplot(5, 5, i+1)
  plt.xticks([])
  plt.yticks([])
  plt.grid(False)
  plt.imshow(image, cmap=plt.cm.binary)
  plt.xlabel(class_names[label])
  i += 1

plt.show()

In [0]:
# Setup Layers
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28, 1)),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

BATCH_SIZE = 32
train_dataset = train_dataset.repeat().shuffle(num_train_examples).batch(BATCH_SIZE)
test_dataset = test_dataset.batch(BATCH_SIZE)

model.fit(train_dataset, epochs=5, steps_per_epoch=math.ceil(num_train_examples/BATCH_SIZE))

In [0]:
test_loss, test_accuracy = model.evaluate(test_dataset, steps=math.ceil(num_test_examples / 32))
print('Accuracy on test dataset = ', test_accuracy)

In [0]:
# Predict
for test_images, test_labels in test_dataset.take(1):
  test_images = test_images.numpy()
  test_labels = test_labels.numpy()
  predictions = model.predict(test_images)

predictions.shape

In [0]:
predictions[0]

In [0]:
# Give index of max value in predictions
np.argmax(predictions[0])

In [0]:
# Verify the prediction with test label
test_labels[0]

## Exercises
  - Set training epochs to 1
  - Notice accuracy changes by setting no. of neurons in Dense layer following Flat one to low values like 10 in ranges up to 512
  - Add additional layers between final Dense layers and Flatten layers and experiment with different units in these layers
  - Don't normalize and see the effect